In [2]:
import vtk
import numpy as np
from vtk.util import numpy_support
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi

C:\Users\ahmed\AppData\Local\Temp\ipykernel_23368\1881236227.py:5: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.4)
  from scipy.spatial import Voronoi


In [3]:
object_path = "./assets/cat1.obj"

reader = vtk.vtkOBJReader()
reader.SetFileName(object_path)
reader.Update()

# --- Step 2: Create a mapper ---
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())

# --- Step 3: Create an actor ---
actor = vtk.vtkActor()
actor.SetMapper(mapper)


In [4]:
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

In [5]:
# --- Step 5: Add actor and background ---
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background

# --- Step 6: Start rendering ---
render_window.Render()
interactor.Start()

In [6]:
# --- Step 1: Load vertices from file ---
coordinates_file_path = "./assets/Objet3D.txt"

coordinates = []
with open(coordinates_file_path, "r") as file:
    for line in file:
        if line.startswith("v "):  # Only process vertex lines
            _, x, y, z = line.strip().split()
            coordinates.append([float(x), float(y), float(z)])

points = np.array(coordinates)
print(f"Loaded {len(points)} points.")

# --- Step 2: Convert numpy array to VTK points ---
vtk_points = vtk.vtkPoints()
for p in points:
    vtk_points.InsertNextPoint(p.tolist())

Loaded 4008 points.


In [7]:
vor = Voronoi(points)

# --- Step 3: Create VTK PolyData for Voronoi edges ---
vtk_voronoi_points = vtk.vtkPoints()
for v in vor.vertices:
    vtk_voronoi_points.InsertNextPoint(v.tolist())

voronoi_polydata = vtk.vtkPolyData()
voronoi_polydata.SetPoints(vtk_voronoi_points)

voronoi_lines = vtk.vtkCellArray()
for ridge in vor.ridge_vertices:
    if -1 not in ridge and len(ridge) >= 2:
        for i in range(len(ridge) - 1):
            line = vtk.vtkLine()
            line.GetPointIds().SetId(0, ridge[i])
            line.GetPointIds().SetId(1, ridge[i + 1])
            voronoi_lines.InsertNextCell(line)

voronoi_polydata.SetLines(voronoi_lines)

In [8]:
voronoi_mapper = vtk.vtkPolyDataMapper()
voronoi_mapper.SetInputData(voronoi_polydata)

voronoi_actor = vtk.vtkActor()
voronoi_actor.SetMapper(voronoi_mapper)
voronoi_actor.GetProperty().SetColor(1, 0, 0)   # red lines
voronoi_actor.GetProperty().SetLineWidth(1.5)

In [ ]:

# --- Step 5: Create Delaunay triangulation (VTK native) ---
vtk_points = vtk.vtkPoints()
for p in points:
    vtk_points.InsertNextPoint(p.tolist())

input_polydata = vtk.vtkPolyData()
input_polydata.SetPoints(vtk_points)

delaunay = vtk.vtkDelaunay3D()
delaunay.SetInputData(input_polydata)
delaunay.Update()

delaunay_surface_filter = vtk.vtkDataSetSurfaceFilter()
delaunay_surface_filter.SetInputConnection(delaunay.GetOutputPort())
delaunay_surface_filter.Update()

delaunay_mapper = vtk.vtkPolyDataMapper()
delaunay_mapper.SetInputConnection(delaunay_surface_filter.GetOutputPort())

delaunay_actor = vtk.vtkActor()
delaunay_actor.SetMapper(delaunay_mapper)
delaunay_actor.GetProperty().SetColor(0, 0, 1)  # blue surface
delaunay_actor.GetProperty().SetOpacity(0.3)

In [ ]:
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

renderer.AddActor(voronoi_actor)
renderer.AddActor(delaunay_actor)
renderer.SetBackground(1, 1, 1)

render_window.SetSize(800, 600)
render_window.Render()
interactor.Start()